In [17]:
import pandas as pd
from sqlalchemy import text
from sqlalchemy import create_engine
import pyodbc

In [18]:
pyodbc.drivers()


['SQL Server',
 'Microsoft Access Driver (*.mdb, *.accdb)',
 'Microsoft Excel Driver (*.xls, *.xlsx, *.xlsm, *.xlsb)',
 'Microsoft Access Text Driver (*.txt, *.csv)',
 'Microsoft Access dBASE Driver (*.dbf, *.ndx, *.mdx)',
 'ODBC Driver 17 for SQL Server',
 'ODBC Driver 18 for SQL Server']

In [19]:

conn_str = (
    "DRIVER={ODBC Driver 18 for SQL Server};"
    "SERVER=localhost;"
    "DATABASE=statsbomb;"
    "Trusted_Connection=yes;"
    "TrustServerCertificate=yes;"
)

conn = pyodbc.connect(conn_str)


In [20]:
cursor = conn.cursor()
cursor.execute("SELECT 1")
print(cursor.fetchone())


(1,)


In [21]:
cursor.execute("""
SELECT TABLE_SCHEMA, TABLE_NAME
FROM INFORMATION_SCHEMA.TABLES
WHERE TABLE_SCHEMA = 'dim'
""")
cursor.fetchall()


[('dim', 'dim_team'),
 ('dim', 'dim_player'),
 ('dim', 'dim_competition'),
 ('dim', 'dim_season'),
 ('dim', 'dim_match'),
 ('dim', 'dim_shot_attributes'),
 ('dim', 'dim_date')]

In [22]:
conn_str = (
    "DRIVER={ODBC Driver 18 for SQL Server};"
    "SERVER=localhost;"
    "DATABASE=statsbomb;"
    "Trusted_Connection=yes;"
    "TrustServerCertificate=yes;"
)

engine = create_engine(
    "mssql+pyodbc://",
    creator=lambda: pyodbc.connect(conn_str)
)


In [23]:
pd.read_sql("SELECT DB_NAME() AS db", engine)

c:\Users\Βασιλης\statsbomb_project\.venv\Lib\site-packages\pandas\io\sql.py:1648: SAWarning: Unrecognized server version info '17.0.1000.7'.  Some SQL Server features may not function properly.
  con = self.exit_stack.enter_context(con.connect())


,db
0,statsbomb


In [24]:

with engine.connect() as conn:
    result = conn.execute(text("SELECT 1"))
    print(result.fetchone())


(1,)


Start of csv loading

In [26]:
df_team = pd.read_csv(
    "../data/processed/dim_team.csv",
    dtype={
        "team_id": "int64",
        "team_name": "string"
    }
)

df_team


,team_id,team_name
0,22,Leicester City
1,28,AFC Bournemouth
2,27,West Bromwich Albion
3,41,Sunderland
4,37,Newcastle United
5,59,Aston Villa
6,29,Everton
7,31,Crystal Palace
8,23,Watford
9,1,Arsenal


In [29]:

with engine.begin() as conn:
    conn.execute(text("DELETE FROM dim.dim_team"))


In [30]:
df_team.to_sql(
    name="dim_team",
    schema="dim",
    con=engine,
    if_exists="append",
    index=False,
    method="multi"
)


20

Dim player

In [ ]:

df_player = pd.read_csv(
    "../data/processed/dim_player.csv",
    dtype={
        "player_id": "int64",
        "player_name": "string",
        "position_name": "string"
    }
)

,player_id,player_name,position_name
0,2956,Bertrand Isidore Traoré,Center Forward
1,2966,Stéphane Sessègnon,Right Midfield
2,2988,Memphis Depay,Left Wing
3,3037,Mousa Sidi Yaya Dembélé,Left Defensive Midfield
4,3041,Ciaran Clark,Left Center Back
...,...,...,...
544,75899,Kevin Toner,Left Center Back
545,128703,Dionatan do Nascimento Teixeira,Left Center Back
546,128704,Emmanuel Emenike,Center Forward
547,129558,Tony Hibbert,Right Back


In [33]:
with engine.begin() as conn:
    conn.execute(text("DELETE FROM dim.dim_player"))

In [34]:
df_player.to_sql(
    name="dim_player",
    schema="dim",
    con=engine,
    if_exists="append",
    index=False,
    method="multi"
)


549

dim_competition

In [36]:
df_comp = pd.read_csv(
    "../data/processed/dim_competition.csv",
    dtype={
        "competition_id": "int64",
        "competition_name": "string",
        "country_name": "string"
    }
)

df_comp

,competition_id,competition_name,country_name
0,2,Premier League,England


In [37]:
with engine.begin() as conn:
    conn.execute(text("DELETE FROM dim.dim_competition"))

In [38]:
df_comp.to_sql(
    name="dim_competition",
    schema="dim",
    con=engine,
    if_exists="append",
    index=False,
    method="multi"
)


1

dim_season

In [39]:
df_season = pd.read_csv(
    "../data/processed/dim_season.csv",
    dtype={
        "season_id": "int64",
        "season_name": "string"
    }
)

df_season

,season_id,season_name
0,27,2015/2016
1,44,2003/2004


In [40]:
with engine.begin() as conn:
    conn.execute(text("DELETE FROM dim.dim_season"))

In [41]:
df_season.to_sql(
    name="dim_season",
    schema="dim",
    con=engine,
    if_exists="append",
    index=False,
    method="multi"
)


2

dim_match

In [42]:
df_match = pd.read_csv(
    "../data/processed/dim_match.csv",
    dtype={
        "match_id": "int64",
        "match_date": "string",
        "kick_off": "string",
        "home_team_name": "string",
        "away_team_name": "string",
        "home_score": "int64",
        "away_score": "int64",
        "match_week": "int64"
    }
)

df_match.head()

,match_id,match_date,kick_off,home_team_name,away_team_name,home_score,away_score,match_week
0,3754058,2016-01-02,16:00:00.000,Leicester City,AFC Bournemouth,0,0,20
1,3754245,2015-10-17,16:00:00.000,West Bromwich Albion,Sunderland,1,0,9
2,3754136,2015-12-19,18:30:00.000,Newcastle United,Aston Villa,1,1,17
3,3754037,2016-04-30,16:00:00.000,Everton,AFC Bournemouth,2,1,36
4,3754039,2016-02-13,16:00:00.000,Crystal Palace,Watford,1,2,26


In [43]:
df_match["match_date"] = pd.to_datetime(df_match["match_date"]).dt.date
df_match["kick_off"] = pd.to_datetime(df_match["kick_off"]).dt.time


C:\Users\Βασιλης\AppData\Local\Temp\ipykernel_19520\423445720.py:2: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df_match["kick_off"] = pd.to_datetime(df_match["kick_off"]).dt.time


In [44]:
df_team = pd.read_sql(
    "SELECT team_key, team_name FROM dim.dim_team",
    engine
)

df_team


,team_key,team_name
0,1,Leicester City
1,2,AFC Bournemouth
2,3,West Bromwich Albion
3,4,Sunderland
4,5,Newcastle United
5,6,Aston Villa
6,7,Everton
7,8,Crystal Palace
8,9,Watford
9,10,Arsenal


In [45]:
df_match = df_match.merge(
    df_team,
    left_on="home_team_name",
    right_on="team_name",
    how="left"
).rename(columns={"team_key": "home_team_key"}).drop(columns=["team_name"])


In [46]:
df_match = df_match.merge(
    df_team,
    left_on="away_team_name",
    right_on="team_name",
    how="left"
).rename(columns={"team_key": "away_team_key"}).drop(columns=["team_name"])


In [47]:
df_match[["home_team_name", "home_team_key", "away_team_name", "away_team_key"]].head()


,home_team_name,home_team_key,away_team_name,away_team_key
0,Leicester City,1,AFC Bournemouth,2
1,West Bromwich Albion,3,Sunderland,4
2,Newcastle United,5,Aston Villa,6
3,Everton,7,AFC Bournemouth,2
4,Crystal Palace,8,Watford,9


In [49]:
comp_key = pd.read_sql(
    "SELECT competition_key FROM dim.dim_competition WHERE competition_name = 'Premier League'",
    engine
).iloc[0, 0]

season_key = pd.read_sql(
    "SELECT season_key FROM dim.dim_season WHERE season_name = '2015/2016'",
    engine
).iloc[0, 0]

df_match["competition_key"] = comp_key
df_match["season_key"] = season_key


In [50]:
df_match_final = df_match[[
    "match_id",
    "match_date",
    "kick_off",
    "competition_key",
    "season_key",
    "home_team_key",
    "away_team_key",
    "home_team_name",
    "away_team_name",
    "home_score",
    "away_score",
    "match_week"
]]


In [53]:
df_match_final

,match_id,match_date,kick_off,competition_key,season_key,home_team_key,away_team_key,home_team_name,away_team_name,home_score,away_score,match_week
0,3754058,2016-01-02,16:00:00,1,1,1,2,Leicester City,AFC Bournemouth,0,0,20
1,3754245,2015-10-17,16:00:00,1,1,3,4,West Bromwich Albion,Sunderland,1,0,9
2,3754136,2015-12-19,18:30:00,1,1,5,6,Newcastle United,Aston Villa,1,1,17
3,3754037,2016-04-30,16:00:00,1,1,7,2,Everton,AFC Bournemouth,2,1,36
4,3754039,2016-02-13,16:00:00,1,1,8,9,Crystal Palace,Watford,1,2,26
...,...,...,...,...,...,...,...,...,...,...,...,...
375,3754020,2015-08-17,21:00:00,1,1,11,2,Liverpool,AFC Bournemouth,1,0,2
376,3754267,2015-08-15,16:00:00,1,1,9,3,Watford,West Bromwich Albion,0,0,2
377,3754141,2015-08-09,14:30:00,1,1,10,18,Arsenal,West Ham United,0,2,1
378,3754128,2015-08-08,16:00:00,1,1,2,6,AFC Bournemouth,Aston Villa,0,1,1


In [51]:
with engine.begin() as conn:
    conn.execute(text("DELETE FROM dim.dim_match"))


In [54]:
df_match_final.to_sql(
    name="dim_match",
    schema="dim",
    con=engine,
    if_exists="append",
    index=False,
    method="multi",
    chunksize=100
)


380

dim_shots

In [ ]:
df_shot = pd.read_csv(
    "../data/processed/fact_shot.csv",
    dtype={
        "shot_outcome": "string",
        "shot_body_part": "string",
        "shot_type": "string",
        "shot_technique": "string",
        "shot_first_time": "boolean",
        "shot_one_on_one": "boolean"
    }
)

df_shot.head()

,event_id,shot_statsbomb_xg,shot_outcome,shot_body_part,shot_type,shot_technique,shot_first_time,shot_one_on_one,end_x,end_y,end_z
0,72596ffe-393e-4a75-82c6-5fb82ae36d4d,0.020838,Blocked,Right Foot,Open Play,Normal,False,False,109.7,30.1,NaN
1,b2bae775-5dd8-45bf-9b84-191c7849f707,0.034462,Blocked,Right Foot,Open Play,Normal,False,False,114.5,52.6,NaN
2,7c4227e5-0759-4633-93ea-bbbe291bbd72,0.035481,Off T,Left Foot,Open Play,Overhead Kick,True,False,120.0,43.3,3.6
3,8e3a6f10-64e4-49bf-a157-16b15b552713,0.383711,Off T,Head,Open Play,Normal,False,False,120.0,46.0,1.2
4,3a5364b3-0b99-4271-a451-0a3033ad195d,0.036740,Wayward,Right Foot,Open Play,Half Volley,False,False,103.9,40.1,NaN


In [58]:
shot_attr_cols = [
    "shot_outcome",
    "shot_body_part",
    "shot_type",
    "shot_technique",
    "shot_first_time",
    "shot_one_on_one"
]

df_attr = df_shot[shot_attr_cols]


In [59]:
df_attr["shot_first_time"] = df_attr["shot_first_time"].astype("int")
df_attr["shot_one_on_one"] = df_attr["shot_one_on_one"].astype("int")


C:\Users\Βασιλης\AppData\Local\Temp\ipykernel_19520\3454193309.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_attr["shot_first_time"] = df_attr["shot_first_time"].astype("int")
C:\Users\Βασιλης\AppData\Local\Temp\ipykernel_19520\3454193309.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_attr["shot_one_on_one"] = df_attr["shot_one_on_one"].astype("int")


In [60]:
df_attr = df_attr.drop_duplicates().reset_index(drop=True)

df_attr


,shot_outcome,shot_body_part,shot_type,shot_technique,shot_first_time,shot_one_on_one
0,Blocked,Right Foot,Open Play,Normal,0,0
1,Off T,Left Foot,Open Play,Overhead Kick,1,0
2,Off T,Head,Open Play,Normal,0,0
3,Wayward,Right Foot,Open Play,Half Volley,0,0
4,Post,Right Foot,Open Play,Normal,0,1
...,...,...,...,...,...,...
225,Post,Left Foot,Open Play,Half Volley,0,0
226,Wayward,Left Foot,Open Play,Backheel,1,0
227,Wayward,Left Foot,Open Play,Overhead Kick,0,0
228,Goal,Left Foot,Open Play,Backheel,1,0


In [62]:
with engine.begin() as conn:
    conn.execute(text("DELETE FROM dim.dim_shot_attributes"))

In [63]:
df_attr.to_sql(
    name="dim_shot_attributes",
    schema="dim",
    con=engine,
    if_exists="append",
    index=False,
    method="multi",
    chunksize=100
)


230

dim_pass_attributes

In [ ]:
df_pass = pd.read_csv(
    "../data/processed/fact_pass.csv",
    dtype={
        "pass_height": "string",
        "pass_type": "string",
        "pass_outcome": "string"
    }
)

,event_id,end_x,end_y,pass_length,pass_angle,pass_height,pass_type,pass_outcome,pass_recipient
0,2ca23eea-a984-47e4-8243-8f00880ad1c9,60.4,43.6,3.551056,1.740575,Ground Pass,Kick Off,<NA>,Joshua King
1,0fee7719-7e69-49c5-be81-3f2b77da604e,48.0,41.7,12.544720,-2.989549,Ground Pass,<NA>,<NA>,Andrew Surman
2,6362aa69-892f-4d11-8644-21a680ea7c66,37.5,76.1,35.966790,1.867047,Ground Pass,<NA>,<NA>,Adam Smith
3,56da36e4-8b0d-4596-ba46-1d944c3d3f04,27.4,58.1,19.345800,-2.120081,Ground Pass,<NA>,<NA>,Simon Francis
4,bcfea2e3-9736-4975-be28-ef2c9d693fa7,35.1,77.8,15.890248,1.064920,Ground Pass,<NA>,<NA>,Adam Smith


In [67]:
df_attr = df_pass[[
    "pass_height",
    "pass_type",
    "pass_outcome"
]].drop_duplicates().reset_index(drop=True)


In [69]:
with engine.begin() as conn:
    conn.execute(text("DELETE FROM dim.dim_pass_attributes"))

In [70]:
df_attr.to_sql(
    name="dim_pass_attributes",
    schema="dim",
    con=engine,
    if_exists="append",
    index=False,
    method="multi",
    chunksize=100
)


102

FACT_EVENT

In [72]:
df_event = pd.read_csv(
    "../data/processed/fact_event.csv",
    dtype={
        "event_id": "string",
        "match_id": "int64",
        "team_id": "int64",
        "player_id": "Int64",   
        "period": "int64",
        "minute": "int64",
        "second": "int64",
        "event_type": "string",
        "event_category": "string",
        "possession": "int64",
        "possession_team_id": "int64",
        "under_pressure": "boolean",
        "x": "float",
        "y": "float",
        "is_pass": "boolean",
        "is_shot": "boolean",
        "is_carry": "boolean",
        "is_pressure": "boolean"
    }
)

In [73]:
bool_cols = [
    "under_pressure",
    "is_pass",
    "is_shot",
    "is_pressure",
    "is_carry"
]

for c in bool_cols:
    df_event[c] = df_event[c].astype("int")


In [74]:
df_match = pd.read_sql(
    "SELECT match_key, match_id FROM dim.dim_match",
    engine
)

df_event = df_event.merge(
    df_match,
    on="match_id",
    how="left"
)


In [75]:
df_team = pd.read_sql(
    "SELECT team_key, team_id FROM dim.dim_team",
    engine
)

df_event = df_event.merge(
    df_team,
    on="team_id",
    how="left"
)

df_event = df_event.merge(
    df_team.rename(columns={
        "team_key": "possession_team_key",
        "team_id": "possession_team_id"
    }),
    on="possession_team_id",
    how="left"
)


In [76]:
df_player = pd.read_sql(
    "SELECT player_key, player_id FROM dim.dim_player",
    engine
)

df_event = df_event.merge(
    df_player,
    on="player_id",
    how="left"
)


In [77]:
df_event_final = df_event[[
    "event_id",
    "match_key",
    "team_key",
    "player_key",
    "period",
    "minute",
    "second",
    "event_type",
    "event_category",
    "possession",
    "possession_team_key",
    "under_pressure",
    "x",
    "y",
    "is_pass",
    "is_shot",
    "is_pressure",
    "is_carry",
]]


In [79]:
with engine.begin() as conn:
    conn.execute(text("DELETE FROM fact.fact_event"))

In [81]:
df_event_final.to_sql(
    name="fact_event",
    schema="fact",
    con=engine,
    if_exists="append",
    index=False,
    method="multi",
    chunksize=100
)

1313783

fact_shot

In [83]:
df_shot = pd.read_csv(
    "../data/processed/fact_shot.csv",
    dtype={
        "event_id": "string",
        "shot_statsbomb_xg": "float",
        "shot_outcome": "string",
        "shot_body_part": "string",
        "shot_type": "string",
        "shot_technique": "string",
        "shot_first_time": "boolean",
        "shot_one_on_one": "boolean",
        "end_x": "float",
        "end_y": "float",
        "end_z": "float"
    }
)

In [84]:
df_shot["shot_first_time"] = df_shot["shot_first_time"].astype("int")
df_shot["shot_one_on_one"] = df_shot["shot_one_on_one"].astype("int")


In [85]:
df_shot_attr = pd.read_sql(
    """
    SELECT
        shot_attr_key,
        shot_outcome,
        shot_body_part,
        shot_type,
        shot_technique,
        shot_first_time,
        shot_one_on_one
    FROM dim.dim_shot_attributes
    """,
    engine
)

df_shot = df_shot.merge(
    df_shot_attr,
    on=[
        "shot_outcome",
        "shot_body_part",
        "shot_type",
        "shot_technique",
        "shot_first_time",
        "shot_one_on_one"
    ],
    how="left"
)


In [87]:
df_event = pd.read_sql(
    """
    SELECT
        event_id,
        event_key,
        match_key,
        team_key,
        player_key
    FROM fact.fact_event
    """,
    engine
)

df_shot = df_shot.merge(
    df_event,
    on="event_id",
    how="left"
)


In [89]:
df_shot_final = df_shot[[
    "event_id",
    "event_key",
    "match_key",
    "team_key",
    "player_key",
    "shot_attr_key",
    "shot_statsbomb_xg",
    "end_x",
    "end_y",
    "end_z"
]]


In [90]:
with engine.begin() as conn:
    conn.execute(text("DELETE FROM fact.fact_shot"))

In [91]:
df_shot_final.to_sql(
    name="fact_shot",
    schema="fact",
    con=engine,
    if_exists="append",
    index=False,
    method="multi",
    chunksize=100
)


9908

In [93]:
pd.read_sql(
    "SELECT TOP 10 fs.event_id, fe.event_type, fs.shot_statsbomb_xg FROM fact.fact_shot fs JOIN fact.fact_event fe ON fs.event_key = fe.event_key",
    engine
)


,event_id,event_type,shot_statsbomb_xg
0,7db239ff-9ff2-4e31-bb58-acb3a1cba52e,Shot,0.245253
1,3ca8bc18-6083-4d5a-afc1-4dc20de42e43,Shot,0.042902
2,ae5dfce5-95e6-47b1-b2e3-abc11d7d59f2,Shot,0.012095
3,39ba03e1-5445-4735-a38e-c0b8330d9883,Shot,0.023960
4,4a791b58-400d-412b-89e9-7d6a8dc6d454,Shot,0.044721
5,a2904741-25d0-4824-aa20-1aed74d26389,Shot,0.028252
6,0deef0c6-856b-469c-bfaa-15264d794cd2,Shot,0.020842
7,ca9b925c-13c5-4a4c-bcbb-d5c48ed89d91,Shot,0.039429
8,24f7f6d4-bbb0-45da-8414-2cb58af7d8f4,Shot,0.008613
9,a9326c0b-1e10-43f1-8d0d-de63ca8a6114,Shot,0.057364


fact_pass

In [94]:
df_pass = pd.read_csv(
    "../data/processed/fact_pass.csv",
    dtype={
        "event_id": "string",
        "end_x": "float",
        "end_y": "float",
        "pass_length": "float",
        "pass_angle": "float",
        "pass_height": "string",
        "pass_type": "string",
        "pass_outcome": "string",
        "pass_recipient": "string"
    }
)

In [95]:
df_pass_attr = pd.read_sql(
    """
    SELECT
        pass_attr_key,
        pass_height,
        pass_type,
        pass_outcome
    FROM dim.dim_pass_attributes
    """,
    engine
)

df_pass = df_pass.merge(
    df_pass_attr,
    on=["pass_height", "pass_type", "pass_outcome"],
    how="left"
)


In [97]:
df_event = pd.read_sql(
    """
    SELECT
        event_id,
        event_key,
        match_key,
        team_key,
        player_key AS passer_player_key
    FROM fact.fact_event
    WHERE is_pass = 1
    """,
    engine
)

df_pass = df_pass.merge(
    df_event,
    on="event_id",
    how="left"
)


In [99]:
df_players = pd.read_sql(
    "SELECT player_key, player_name FROM dim.dim_player",
    engine
)

df_pass = df_pass.merge(
    df_players,
    left_on="pass_recipient",
    right_on="player_name",
    how="left"
).rename(columns={"player_key": "receiver_player_key"}).drop(columns=["player_name"])


In [101]:
df_pass[["pass_recipient", "receiver_player_key"]]


,pass_recipient,receiver_player_key
0,Joshua King,83.0
1,Andrew Surman,81.0
2,Adam Smith,327.0
3,Simon Francis,176.0
4,Adam Smith,327.0
...,...,...
368614,Gylfi Þór Sigurðsson,NaN
368615,Radamel Falcao García Zárate,NaN
368616,<NA>,NaN
368617,Éderzito António Macedo Lopes,NaN


In [102]:
df_pass_final = df_pass[[
    "event_id",
    "event_key",
    "match_key",
    "team_key",
    "passer_player_key",
    "receiver_player_key",
    "pass_attr_key",
    "end_x",
    "end_y",
    "pass_length",
    "pass_angle"
]]


In [103]:
with engine.begin() as conn:
    conn.execute(text("DELETE FROM fact.fact_pass"))

In [104]:
df_pass_final.to_sql(
    name="fact_pass",
    schema="fact",
    con=engine,
    if_exists="append",
    index=False,
    method="multi",
    chunksize=100
)


368619